In [2]:
# Cell 1: Setup and Imports
import xarray as xr
import numpy as np
import os

In [18]:
from pathlib import Path

p = Path(r"C:\Users\lytten\programming\climarisc\data\climate_raw\usa_era5_land_hourly\1981\test.nc")
print(p)                # sanity check the string
print(p.exists())       # should be True
print(p.is_file())      # should be True
print(p.stat().st_size) # size in bytes if it exists


C:\Users\lytten\programming\climarisc\data\climate_raw\usa_era5_land_hourly\1981\test.nc
True
True
263461086


In [4]:
import xarray as xr

# Open the NetCDF file
ds = xr.open_dataset(r"C:\Users\lytten\programming\climarisc\data\climate_raw\usa_era5_land_hourly\1981\era5_land_usa_1981_01.nc")

# Show the dataset structure
print(ds)

#


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'cfgrib']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

# Processing ERA5-Land Climate Data - 1981 Test Case

**Goal:** To process the raw hourly ERA5-Land data for 1981 into analysis-ready, yearly stressor metrics that align with our crop yield data.

In [ ]:
# Cell 2: Load Hourly Data for 1981 (Using the CORRECT cfgrib engine)

# Path to the downloaded files for the year 1981
DATA_PATH = '../data/climate_raw/usa_era5_land_hourly/1981/test.nc'

try:
    # Use open_mfdataset to combine the 12 monthly files along the time dimension
    # FINAL CORRECTION: Using engine='cfgrib' as indicated by the file's own metadata.
    ds_hourly = xr.open_mfdataset(DATA_PATH, engine='cfgrib')
    
    print("Successfully loaded 12 monthly files for 1981.")
    print(ds_hourly)
except Exception as e:
    print(f"Could not load data. Please ensure the files for 1981 have finished downloading.")
    print(f"Error: {e}")

skipping corrupted Message
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\messages.py", line 274, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\messages.py", line 341, in message_from_file
    return Message.from_file(file, offset, **kwargs)
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\messages.py", line 97, in from_file
    codes_id = eccodes.codes_grib_new_from_file(file)
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\gribapi\gribapi.py", line 409, in grib_new_from_file
    GRIB_CHECK(err)
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\gribapi\gribapi.py", line 226, in GRIB_CHECK
    errors.raise_grib_error(errid)
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\gribapi\errors.py", line 381, in raise_grib_error
    raise ERROR_MAP[e

## 1. Aggregate from Hourly to Daily

We will convert the hourly data into key daily metrics:
- **Max Temperature (Tmax):** The highest temperature of the day.
- **Min Temperature (Tmin):** The lowest temperature of the day.
- **Total Precipitation (Precip):** The total rainfall for the day.

In [ ]:
# Cell 3: Resample to Daily Data

# The 't2m' variable is '2m_temperature' in Kelvin.
# We resample by taking the daily max and min, and convert to Celsius.
daily_tmax = ds_hourly['t2m'].resample(time='1D').max() - 273.15
daily_tmin = ds_hourly['t2m'].resample(time='1D').min() - 273.15

# The 'tp' variable is 'total_precipitation' in meters.
# It's a cumulative value, so we resample by taking the daily sum and convert to mm.
daily_precip = ds_hourly['tp'].resample(time='1D').sum() * 1000

# Combine these into a new, clean daily dataset
ds_daily = xr.Dataset({
    'tmax': daily_tmax,
    'tmin': daily_tmin,
    'precip': daily_precip
})

print("Resampled to daily data:")
print(ds_daily)

## 2. Calculate Yearly Growing-Season Metrics

We'll assume a growing season for maize in the US is from April to September (months 4-9) and calculate summary statistics for that period.

In [ ]:
# Cell 4: Calculate Yearly Stress Metrics

# Select only the months in the growing season
growing_season = ds_daily.where(ds_daily['time.month'].isin(range(4, 10)), drop=True)

# Calculate yearly metrics
# 1. Total growing season precipitation
total_precip_1981 = growing_season['precip'].sum(dim='time')

# 2. Average maximum temperature during the growing season
avg_tmax_1981 = growing_season['tmax'].mean(dim='time')

# 3. Number of extreme heat days (e.g., days > 30°C)
extreme_heat_days_1981 = growing_season['tmax'].where(growing_season['tmax'] > 30).count(dim='time')

# Combine into a yearly dataset
ds_yearly = xr.Dataset({
    'total_precip': total_precip_1981,
    'avg_tmax': avg_tmax_1981,
    'extreme_heat_days': extreme_heat_days_1981
})

print("Calculated yearly metrics for 1981 growing season:")
print(ds_yearly)

## 3. Align Climate Grid to Yield Grid

The ERA5 data is on a 0.1° grid, but our yield data is on a 0.5° grid. We must resample the climate data to match the yield data's coordinates.

In [ ]:
# Cell 5: Regrid Climate Data and Save

# First, load a sample of the yield data to use as a grid template
YIELD_PATH = '../data/maize/yield_1981.nc4'
yield_grid = xr.open_dataset(YIELD_PATH)

# Use .interp_like() to resample our yearly climate data to the yield grid
print("Regridding climate data to match yield data...")
ds_yearly_aligned = ds_yearly.interp_like(yield_grid)

print("\nFinal, aligned dataset for 1981:")
print(ds_yearly_aligned)

# --- Save the final processed file ---
output_dir = '../data/climate_processed/'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'climate_yearly_usa_1981.nc')
ds_yearly_aligned.to_netcdf(output_file)

print(f"\nSuccessfully saved processed 1981 climate data to: {output_file}")